# XArray Basics

This notebook exists to demonstrate some of the basic features and capabilities that Xarray provides out-of-box. It also contains links to documentation and further reading that digs into some of the more advanced features. This notebook may be expanded in the future to provide guidance on some of those features as well


## Why Use Xarray?

[From the article](http://xarray.pydata.org/en/stable/why-xarray.html):

> Multi-dimensional (a.k.a. N-dimensional, ND) arrays (sometimes called “tensors”) are an essential part of computational science. They are encountered in a wide range of fields, including physics, astronomy, geoscience, bioinformatics, engineering, finance, and deep learning. In Python, NumPy provides the fundamental data structure and API for working with raw ND arrays. However, real-world datasets are usually more than just raw numbers; they have labels which encode information about how the array values map to locations in space, time, etc.

> Xarray doesn’t just keep track of labels on arrays – it uses them to provide a powerful and concise interface.

There are several other existing libraries that provide functionality similar to xarray (netCDF4-python, iris, UV-CDAT), but Xarray shines through by building on some of the best features of numpy and pandas to create objects that allow you to build arbitrarily-multidimensional data objects rich in metadata. Xarray is performant, well-documented, and is widely adopted among many scientific communities. These are a few of the reasons why xarray has been chosen for use in this project.  

Also, integration with CF conventions, ease of working with files, automatic time conversion & other built-in methods, and a few built-in plotting routines.

# Examples

In this next section we will demonstrate / experiment with some of the basic features that Xarray offers.


## 1. Creating a simple dataset with UTF timestamp saved as a long data type

Creating a dataset from scratch with xarray is pretty easy and can be done in a number of ways. The following example shows just one way of doing it.

In [1]:
import datetime
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
# create dummy dataframe
base_time = 1498867200
times = [1498867200, 1498867260, 1498867320]
heights = [1, 2, 4]

# create dataset
ds_dict = {
    'time'  : xr.DataArray(
                data   = np.array([1498867200, 1498867260, 1498867320], np.int64),
                dims   = ['time'],
                attrs  = {
                    'units'    : 'seconds since 1970-01-01T00:00:00'
                    }
                ),
    'height'  : xr.DataArray(
                data   = np.array([1000, 2000, 8000], np.float32),
                dims   = ['height'],
                attrs  = {
                    'units'    : 'millimeters'
                    }
                ),
    'SWdown': xr.DataArray(
                data   = np.array([[1, 2, 5], [3, 4, 7], [5, -9999, -9999]], np.float32),
                dims   = ['time', 'height'],
                attrs  = {
                    '_FillValue': -9999,
                    'data_type' : 'float',
                    'long_name' : 'Shortwave Downwelling Radiation',
                    'units'     : 'W/m2',
                    'comment'   : 'Short-Wave Downwelling Radiation measured at ground level. Short-wave radiation (visible light) comes from the sun and contains much more energy than Long-wave radiation.'
                    }
                ),
    'LWdown': xr.DataArray(
                data   = np.random.random(3),   # enter data here
                dims   = ['time'],
                attrs  = {
                    '_FillValue': -9999,
                    'data_type' : 'float',
                    'long_name' : 'Longwave Downwelling Radiation',
                    'units'     : 'W/m2'
                    }
                )
            }
ds = xr.Dataset(ds_dict, attrs = {'example_attr': 'this is a global attribute'})

ds

<xarray.Dataset>
Dimensions:  (height: 3, time: 3)
Coordinates:
  * time     (time) int64 1498867200 1498867260 1498867320
  * height   (height) float32 1000.0 2000.0 8000.0
Data variables:
    SWdown   (time, height) float32 1.0 2.0 5.0 3.0 4.0 7.0 5.0 -9999.0 -9999.0
    LWdown   (time) float64 0.1704 0.8123 0.04896
Attributes:
    example_attr:  this is a global attribute

## Step 2: Convert the dataset over to standard time format

In [3]:
ds['time']

<xarray.DataArray 'time' (time: 3)>
array([1498867200, 1498867260, 1498867320])
Coordinates:
  * time     (time) int64 1498867200 1498867260 1498867320
Attributes:
    units:    seconds since 1970-01-01T00:00:00

In [4]:
# One method to decode the raw data according to CF conventions
# is to use the xarray.decode_cf() method. This is pretty simple
# to use, as shown below. Note that the integer times get 
# converted to datetime objects and also that the -9999 values 
# in the SWdown data change to nan.
decoded = xr.decode_cf(ds)
decoded

<xarray.Dataset>
Dimensions:  (height: 3, time: 3)
Coordinates:
  * time     (time) datetime64[ns] 2017-07-01 ... 2017-07-01T00:02:00
  * height   (height) float32 1000.0 2000.0 8000.0
Data variables:
    SWdown   (time, height) float32 ...
    LWdown   (time) float64 ...
Attributes:
    example_attr:  this is a global attribute

In [5]:
# Unfortunately, the decode_cf method has a unfortunate downside; when 
# you use the decode_cf the dataset and its dataarrays get frozen, 
# making it difficult to set values. (It might be possible, but I 
# have not found a way to do this yet). Note how this 
decoded['LWdown'].data[0] = 50
decoded['LWdown'].data

array([0.17036171, 0.8123039 , 0.04895991])

In [6]:
# Fortunately, there is a much simpler alternative to decode_cf. 
# Xarray states in its documentation that it will automatically
# attempt to parse datasets according to CF conventions when writing
# or reading a netCDF file. We can simply save the dataset in a 
# temp file and reopen it to have XArray decode it.
ds.to_netcdf("./temp.nc", mode="w")
new_ds = xr.open_dataset("./temp.nc")
new_ds

<xarray.Dataset>
Dimensions:  (height: 3, time: 3)
Coordinates:
  * time     (time) datetime64[ns] 2017-07-01 ... 2017-07-01T00:02:00
  * height   (height) float32 1000.0 2000.0 8000.0
Data variables:
    SWdown   (time, height) float32 ...
    LWdown   (time) float64 ...
Attributes:
    example_attr:  this is a global attribute

In [7]:
# With this new dataset we can set values, add attributes, etc.
new_ds["LWdown"].data[0] = -1.4132
new_ds["LWdown"].attrs['comment'] = "Long-Wave Downwelling Radiation measured at ground level. Long-wave radiation (infrared light) is the  and contains much more energy than Long-wave radiation."
new_ds["LWdown"]

<xarray.DataArray 'LWdown' (time: 3)>
array([-1.4132  ,  0.812304,  0.04896 ])
Coordinates:
  * time     (time) datetime64[ns] 2017-07-01 ... 2017-07-01T00:02:00
Attributes:
    data_type:  float
    long_name:  Longwave Downwelling Radiation
    units:      W/m2
    comment:    Long-Wave Downwelling Radiation measured at ground level. Lon...

In [8]:
# NOT WORKING: CURRENTLY GETTING IMPORT ERROR(S) FOR PARSE_CF
# A third alternative for parsing the raw data according to
# CF conventions is to use metpy. Metpy uses a dataset accessor
# to register a "metpy" namespace under datasets/dataarrays.
# One of the functions built in to this namespace is for 
# parsing the dataset according to CF conventions. See below:
import metpy  # Any import of metpy will register the accessors
# met_ds = ds.metpy.parse_cf()
# met_ds

In [9]:
# Note that just by importing metpy in the cell above we gain
# access to a whole suite of attributes and methods under the 
# metpy attribute of a dataset or dataarray:
ds.metpy?

Type:        MetPyDatasetAccessor
String form: <metpy.xarray.MetPyDatasetAccessor object at 0x7fdcf14aabe0>
File:        ~/opt/anaconda3/envs/mhkit_act_env/lib/python3.8/site-packages/metpy/xarray.py
Docstring:  
Provide custom attributes and methods on XArray Datasets for MetPy functionality.

This accessor provides parsing of CF metadata and unit-/coordinate-type-aware selection.

    >>> import xarray as xr
    >>> from metpy.cbook import get_test_data
    >>> ds = xr.open_dataset(get_test_data('narr_example.nc', False)).metpy.parse_cf()
    >>> print(ds['crs'].item())
    Projection: lambert_conformal_conic


In [10]:
ds = xr.open_dataset('http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/realtime/248p1_rt.nc')

In [11]:
ds

<xarray.Dataset>
Dimensions:                     (acmTime: 38646, dwr4Time: 12856, gpsTime: 75719, metaBoundsCount: 2, sourceCount: 11748, sstTime: 77368, syncTime: 12860, upcrossQuantileLength: 23, upcrossTime: 12931, waveFrequency: 100, waveTime: 12860)
Coordinates:
  * waveTime                    (waveTime) datetime64[ns] 2020-04-01T16:00:00...
  * waveFrequency               (waveFrequency) float32 0.024999999 0.03 ... 1.0
  * sstTime                     (sstTime) datetime64[ns] 2020-04-01T16:20:00 ...
  * gpsTime                     (gpsTime) datetime64[ns] 2020-04-01T16:19:31 ...
  * acmTime                     (acmTime) datetime64[ns] 2020-04-01T16:20:00 ...
  * dwr4Time                    (dwr4Time) datetime64[ns] 2020-04-01T16:00:00...
  * upcrossTime                 (upcrossTime) datetime64[ns] 2020-04-01T16:00...
  * syncTime                    (syncTime) datetime64[ns] 2020-04-01T16:00:00...
    metaDeployLatitude          float32 ...
    metaDeployLongitude         float32 ...
Dimensions without coordinates: metaBoundsCount, sourceCount, upcrossQuantileLength
Data variables:
    sourceFilename              (sourceCount) object ...
    waveTimeBounds              (waveTime, metaBoundsCount) datetime64[ns] ...
    waveFlagPrimary             (waveTime) float32 ...
    waveFlagSecondary           (waveTime) float32 ...
    waveHs                      (waveTime) float32 ...
    waveTp                      (waveTime) float32 ...
    waveTa                      (waveTime) float32 ...
    waveDp                      (waveTime) float32 ...
    wavePeakPSD                 (waveTime) float32 ...
    waveTz                      (waveTime) float32 ...
    wavePeakSpread              (waveTime) float32 ...
    waveTi                      (waveTime) float32 ...
    waveTe                      (waveTime) float32 ...
    waveTm13                    (waveTime) float32 ...
    waveTc                      (waveTime) float32 ...
    waveInverseQp               (waveTime) float32 ...
    waveSourceIndex             (waveTime) float64 ...
    waveFrequencyBounds         (waveFrequency, metaBoundsCount) float32 ...
    waveFrequencyFlagPrimary    (waveFrequency) float32 ...
    waveFrequencyFlagSecondary  (waveFrequency) float32 ...
    waveBandwidth               (waveFrequency) float32 ...
    sstTimeBounds               (sstTime, metaBoundsCount) datetime64[ns] ...
    sstFlagPrimary              (sstTime) float32 ...
    sstFlagSecondary            (sstTime) float32 ...
    sstSeaSurfaceTemperature    (sstTime) float32 ...
    sstSourceIndex              (sstTime) float64 ...
    gpsTimeBounds               (gpsTime, metaBoundsCount) datetime64[ns] ...
    gpsLatitude                 (gpsTime) float32 ...
    gpsLongitude                (gpsTime) float32 ...
    gpsSourceIndex              (gpsTime) float64 ...
    acmTimeBounds               (acmTime, metaBoundsCount) datetime64[ns] ...
    acmFlagPrimary              (acmTime) float32 ...
    acmFlagSecondary            (acmTime) float32 ...
    acmSpeed                    (acmTime) float32 ...
    acmSpeedStdDev              (acmTime) float32 ...
    acmDirection                (acmTime) float32 ...
    acmDirectionStdDev          (acmTime) float32 ...
    acmSignalStrength1          (acmTime) float32 ...
    acmSignalStrength2          (acmTime) float32 ...
    acmSignalStrength3          (acmTime) float32 ...
    acmStatus                   (acmTime) float32 ...
    acmSeaSurfaceTemperature    (acmTime) float32 ...
    acmVerticalSpeed            (acmTime) float32 ...
    acmVerticalSpeedStdDev      (acmTime) float32 ...
    acmSourceIndex              (acmTime) float64 ...
    dwr4TimeBounds              (dwr4Time, metaBoundsCount) datetime64[ns] ...
    dwr4Uptime                  (dwr4Time) float64 ...
    dwr4BatteryWeeksOfLife      (dwr4Time) float64 ...
    dwr4EnergyUsed              (dwr4Time) float64 ...
    dwr4SourceIndex             (dwr4Time) float64 ...
    dwr4En

## Combining Datasets

http://xarray.pydata.org/en/stable/combining.html

In [12]:
ds1 = xr.Dataset(ds_dict, attrs = {'example_attr': 'this global attribute is from dataset #1'})

ds2_dict = {
    'time': xr.DataArray(data=np.array([1498867380, 1498867440, 1498867500], np.int64),
                            dims=['time'],
                            attrs  = {'units':'seconds since 1970-01-01T00:00:00'}),
    'height': xr.DataArray(data=np.array([1000, 2000, 8000], np.float32),
                            dims=['height'],
                            attrs={'units':'millimeters'}),
    'depth': xr.DataArray(data=np.array([6, 5, 4, 3, 2, 1, 0, -1, -2], np.float32),
                            dims=['depth'],
                            attrs={'units':'meters below sea level'}),
    'SWdown': xr.DataArray(data=np.array([  [-1, -2, -5], 
                                            [-3, -4, -7], 
                                            [-5, -9999, -9999]], np.float32),
                            dims=['time', 'height'],
                            attrs={'_FillValue': -9999,
                                    'data_type' : 'float',
                                    'long_name' : 'Shortwave Downwelling Radiation',
                                    'units'     : 'W/m2'}),
    'LWdown': xr.DataArray(data=[1.25, 4, 6.75],
                            dims=['time'],
                            attrs={'_FillValue': -9999,
                                    'data_type' : 'float',
                                    'long_name' : 'Longwave Downwelling Radiation',
                                    'units'     : 'W/m2'})
    }

ds2 = xr.Dataset(ds2_dict, attrs = {'example_attr': 'this global attribute is from dataset #2'})
ds2

<xarray.Dataset>
Dimensions:  (depth: 9, height: 3, time: 3)
Coordinates:
  * time     (time) int64 1498867380 1498867440 1498867500
  * height   (height) float32 1000.0 2000.0 8000.0
  * depth    (depth) float32 6.0 5.0 4.0 3.0 2.0 1.0 0.0 -1.0 -2.0
Data variables:
    SWdown   (time, height) float32 -1.0 -2.0 -5.0 -3.0 ... -5.0 -9999.0 -9999.0
    LWdown   (time) float64 1.25 4.0 6.75
Attributes:
    example_attr:  this global attribute is from dataset #2

In [13]:
# The 'merge' option is probably what we want to use for merging datasets.
# Other options include: 'concat', 'combine_first', 'combine_nested', 
# 'combine_by_coords', and 'update' (see the link at the top of this 
# section for more info on each of those)
xr.Dataset.merge(ds2, ds1)

<xarray.Dataset>
Dimensions:  (depth: 9, height: 3, time: 6)
Coordinates:
  * time     (time) int64 1498867200 1498867260 ... 1498867440 1498867500
  * height   (height) float32 1000.0 2000.0 8000.0
  * depth    (depth) float32 6.0 5.0 4.0 3.0 2.0 1.0 0.0 -1.0 -2.0
Data variables:
    SWdown   (time, height) float32 1.0 2.0 5.0 3.0 ... -5.0 -9999.0 -9999.0
    LWdown   (time) float64 0.1704 0.8123 0.04896 1.25 4.0 6.75
Attributes:
    example_attr:  this global attribute is from dataset #2

In [14]:
ds1 = xr.Dataset({"a": ("x", [10, 20, 30, np.nan])}, {"x": [1, 2, 3, 4]})
ds2 = xr.Dataset({"a": ("x", [np.nan, 30, 40, 50])}, {"x": [2, 3, 4, 5]})
print(xr.merge([ds1, ds2], compat="no_conflicts"))

<xarray.Dataset>
Dimensions:  (x: 5)
Coordinates:
  * x        (x) int64 1 2 3 4 5
Data variables:
    a        (x) float64 10.0 20.0 30.0 40.0 50.0


In [15]:
# If you try merging with "no_conflicts" enabled, if there are any non-matching
# NaN values corresponding to a variable in both then a 'MergeError' is raised.
# Note that "no_conflicts" is the default compat argument.
ds1 = xr.Dataset({"a": ("x", [10, 20, 30, np.nan])}, {"x": [1, 2, 3, 4]})
ds2 = xr.Dataset({"a": ("x", [np.nan, 31, 40, 50])}, {"x": [2, 3, 4, 5]})
try:
    merged = xr.merge([ds1, ds2], compat="no_conflicts")
    print(merged)
except Exception as err:
    print(err.__repr__())

MergeError("conflicting values for variable 'a' on objects to be combined. You can skip this check by specifying compat='override'.")


In [16]:
# If you try merging with "override" enabled, if there are any non-matching
# NaN values corresponding to a variable in both then the value from the
# first dataset in the array is taken.
ds1 = xr.Dataset({"a": ("x", [10, 20, 30, np.nan])}, {"x": [1, 2, 3, 4]})
ds2 = xr.Dataset({"a": ("x", [np.nan, 31, 40, 50])}, {"x": [2, 3, 4, 5]})
try:
    merged = xr.merge([ds1, ds2], compat='override')
    print(merged)
except Exception as err:
    print(err.__repr__())

<xarray.Dataset>
Dimensions:  (x: 5)
Coordinates:
  * x        (x) int64 1 2 3 4 5
Data variables:
    a        (x) float64 10.0 20.0 30.0 nan nan


## Resources

Terminology: http://xarray.pydata.org/en/stable/terminology.html / 

Data Structures: http://xarray.pydata.org/en/stable/data-structures.html
* DataArray - an array-like structure with dimensions, attributes, data, and a host of methods
* Dataset - a dict-like collection of DataArray objects

Parsing times:
* Standard: http://xarray.pydata.org/en/stable/time-series.html
* Irregular calendars: http://xarray.pydata.org/en/stable/weather-climate.html#non-standard-calendars-and-dates-outside-the-timestamp-valid-range

File I/O:
* Individual files: http://xarray.pydata.org/en/stable/io.html
* Multi-file datasets: http://xarray.pydata.org/en/stable/generated/xarray.open_mfdataset.html

Testing: http://xarray.pydata.org/en/stable/api.html#testing